In [2]:
import sqlite3  # This library is used for working with SQLite databases in Python.
import pandas as pd  # pandas is used for easy handling of data structures, especially tabular data.
import csv  # The csv module is used to read from and write to CSV files in Python.

In [3]:
# Connecting to the database
conn = sqlite3.connect('S30 ETL Assignment.db')  # Establish a connection to the SQLite database file.
cursor = conn.cursor()  # Creates a cursor object which allows interaction with the database.

In [ ]:
# To Fetch table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# to fetch columns and data of tables 
for (table,) in tables:
    print(f"\nTable: '{table}'")
    
    # To fetch the first 5 rows of data
    query = f"SELECT * FROM {table} LIMIT 5;"
    cursor.execute(query)
    rows = cursor.fetchall()
    
    # To fetch column names of the table presnt 
    column_names = [description[0] for description in cursor.description]
    print(" | ".join(column_names))  # Print column headers
    
    # Print rows
    if rows:
        for row in rows:
            print(row)
    else:
        print("No data available in this table.")



Table: 'sqlite_sequence'
name | seq
('items', 3)
('customers', 100)
('sales', 500)
('orders', 1500)

Table: 'sales'
sales_id | customer_id
(1, 10)
(2, 85)
(3, 42)
(4, 93)
(5, 75)

Table: 'orders'
order_id | sales_id | item_id | quantity
(1, 1, 1, 2)
(2, 1, 2, None)
(3, 1, 3, 1)
(4, 2, 1, 1)
(5, 2, 2, 1)

Table: 'items'
item_id | item_name
(1, 'x')
(2, 'y')
(3, 'z')

Table: 'customers'
customer_id | age
(1, 38)
(2, 43)
(3, 13)
(4, 39)
(5, 18)


In [ ]:
# - query to find the total quantities of each item bought per customer aged 18-35.
# - For each customer, and to get the sum of each item
# - Items with no purchase is  omitted from the final list and the output is stored in SQL_OUTPUT.CSV FILE .
#- using SQL query .
cursor.execute("""
SELECT 
    c.customer_id AS Customer,
    c.age AS Age,
    i.item_name AS Item,
    SUM(o.quantity) AS Quantity
FROM 
    customers c
JOIN 
    sales s ON c.customer_id = s.customer_id
JOIN 
    orders o ON s.sales_id = o.sales_id
JOIN 
    items i ON o.item_id = i.item_id
WHERE 
    c.age BETWEEN 18 AND 35 
    AND o.quantity IS NOT NULL  -- Exclude NULL quantities
GROUP BY  
    c.customer_id, c.age, i.item_name
HAVING 
    SUM(o.quantity) > 0;
""")

# Fetching the data and column names
rows = cursor.fetchall()
# To get the column names from cursor description
columns = [description[0] for description in cursor.description]

# Write the results to a CSV file with semicolon delimiter, 
with open('SQL_output.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(columns)
    for row in rows:
        writer.writerow(row)




In [7]:
# using pandas to exectute the function .
# Saves the result to CSV 
 # Using 'with' to automatically close the connection
with sqlite3.connect('S30 ETL Assignment.db') as conn: 
    # To load all required tables 
    df_customers = pd.read_sql_query("SELECT customer_id, age FROM customers", conn)
    df_items = pd.read_sql_query("SELECT item_id, item_name FROM items", conn)
    df_sales = pd.read_sql_query("SELECT sales_id, customer_id FROM sales", conn)
    df_orders = pd.read_sql_query("SELECT order_id, sales_id, item_id, quantity FROM orders", conn)

df_merged = (df_orders
             .merge(df_sales, on='sales_id', how='inner') 
             .merge(df_customers, on='customer_id', how='inner')  
             .merge(df_items, on='item_id', how='inner')) 

# Filtering customers aged 18-35 and excludeing NULL or 0 quantities
df_filtered = (df_merged[(df_merged['age'] >= 18) & (df_merged['age'] <= 35)]  
               .dropna(subset=['quantity'])  
               .loc[df_merged['quantity'] > 0])  

 
# Aggregate the data by customer and item, summing quantities

df_aggregated = df_filtered.groupby(['customer_id', 'age', 'item_name'], as_index=False)['quantity'].sum()
df_aggregated['quantity'] = df_aggregated['quantity'].astype(int)

df_aggregated.rename(columns={
    'customer_id': 'Customer',
    'age': 'Age',
    'item_name': 'Item',
    'quantity': 'Quantity'
}, inplace=True)

# Display the result
print(df_aggregated)

# Save the result to CSV with ';' delimiter
df_aggregated.to_csv('output_pandas.csv', sep=';', index=False)
print("\nData successfully exported to 'output_pandas.csv'")

    Customer  Age Item  Quantity
0          5   18    x         3
1          5   18    y         3
2          5   18    z         4
3          7   30    x        14
4          7   30    y        19
..       ...  ...  ...       ...
65        97   23    y         2
66        97   23    z         3
67       100   30    x         1
68       100   30    y         1
69       100   30    z         5

[70 rows x 4 columns]

Data successfully exported to 'output_pandas.csv'
